In [4]:
pip install hear21passt


Note: you may need to restart the kernel to use updated packages.


In [5]:
from hear21passt.base import get_basic_model,get_model_passt
import torch
# get the PaSST model wrapper, includes Melspectrogram and the default pre-trained transformer
model = get_basic_model(mode="logits")
print(model.mel) # Extracts mel spectrogram from raw waveforms.
print(model.net) # the transformer network.

# example inference
model.eval()
model = model.cuda()
with torch.no_grad():
    # audio_wave has the shape of [batch, seconds*32000] sampling rate is 32k
    # example audio_wave of batch=3 and 10 seconds
    audio = torch.ones((3, 32000 * 10))*0.5
    audio_wave = audio.cuda()
    logits=model(audio_wave) 



 Loading PASST TRAINED ON AUDISET 


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), ep

/home/teaching/miniconda3/envs/torch-gpu/lib/python3.10/site-packages/hear21passt/models/passt.py:304: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")


In [6]:
from hear21passt.base import get_basic_model
import torch
import torchaudio

# Load your wav files — replace with your actual paths
wav_paths = ['/home/teaching/DANN-pytorch/239946__rainbowdasheve__ghost.wav', '/home/teaching/DANN-pytorch/276470__unkleceeg__birds-chirping-03-short.wav', '/home/teaching/DANN-pytorch/182501__swiftoid__birds-chirping-03.wav']

# Target sample rate for PaSST model
target_sr = 32000

def load_audio(path, target_sr=32000):
    waveform, sr = torchaudio.load(path)  # [channels, samples]
    if sr != target_sr:
        waveform = torchaudio.transforms.Resample(sr, target_sr)(waveform)
    # Convert stereo to mono if needed
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    return waveform.squeeze(0)  # [samples]

# Load and stack audio files into a batch tensor [batch, samples]
audio_list = [load_audio(p, target_sr) for p in wav_paths]

# Make all audio tensors the same length (pad shorter audios)
max_len = max(a.shape[0] for a in audio_list)
audio_padded = []
for a in audio_list:
    if a.shape[0] < max_len:
        padding = torch.zeros(max_len - a.shape[0])
        a = torch.cat([a, padding])
    audio_padded.append(a)

audio_batch = torch.stack(audio_padded)  # shape [batch_size, max_len]

# Get model
model = get_basic_model(mode="logits")
model.eval()
model = model.cuda()

# Move input to GPU
audio_batch = audio_batch.cuda()

with torch.no_grad():
    logits = model(audio_batch)

print("Logits shape:", logits.shape)




 Loading PASST TRAINED ON AUDISET 


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), ep

RuntimeError: The size of tensor a (163) must match the size of tensor b (99) at non-singleton dimension 3

In [ ]:
from hear21passt.base import get_basic_model
import torch
import torchaudio

# Your wav file paths
wav_paths = [
    '/home/teaching/DANN-pytorch/239946__rainbowdasheve__ghost.wav',
    '/home/teaching/DANN-pytorch/276470__unkleceeg__birds-chirping-03-short.wav',
    '/home/teaching/DANN-pytorch/182501__swiftoid__birds-chirping-03.wav'
]

target_sr = 32000
target_length = target_sr * 10  # 10 seconds * 32000 samples/sec = 320,000 samples

def load_audio(path, target_sr=32000):
    waveform, sr = torchaudio.load(path)  # shape: [channels, samples]
    if sr != target_sr:
        waveform = torchaudio.transforms.Resample(sr, target_sr)(waveform)
    # Convert to mono if stereo
    if waveform.shape[0] > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    waveform = waveform.squeeze(0)  # shape: [samples]
    
    # Truncate or pad to target_length
    if waveform.shape[0] > target_length:
        waveform = waveform[:target_length]
    elif waveform.shape[0] < target_length:
        padding = torch.zeros(target_length - waveform.shape[0])
        waveform = torch.cat([waveform, padding])
    return waveform

# Load all audios and stack into batch tensor of shape [batch, samples]
audio_list = [load_audio(p, target_sr) for p in wav_paths]
audio_batch = torch.stack(audio_list)

print(f"Audio batch shape: {audio_batch.shape}")  # Should be [batch_size, 320000]

# Load PaSST model
model = get_basic_model(mode="embeddings")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Move batch to device
audio_batch = audio_batch.to(device)

# Forward pass
with torch.no_grad():
    logits = model(audio_batch)

print(f"Logits shape: {logits.shape}")


Audio batch shape: torch.Size([3, 320000])


 Loading PASST TRAINED ON AUDISET 


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1):

In [8]:
print(logits)

tensor([[-0.2616, -4.3777, -4.6781,  ..., -5.1485, -5.2438, -7.6786],
        [-3.7312, -7.0227, -6.4901,  ..., -6.7203, -8.4598, -6.6869],
        [-2.3853, -7.2230, -7.1111,  ..., -7.8349, -9.6991, -7.2940]],
       device='cuda:0')


In [10]:
from hear21passt.base import get_basic_model
import torch
import librosa

# Your wav file paths
wav_paths = [
    '/home/teaching/DANN-pytorch/239946__rainbowdasheve__ghost.wav',
    '/home/teaching/DANN-pytorch/276470__unkleceeg__birds-chirping-03-short.wav',
    '/home/teaching/DANN-pytorch/182501__swiftoid__birds-chirping-03.wav'
]

target_sr = 32000
target_length = target_sr * 10  # 10 seconds * 32000 samples/sec = 320,000 samples

def load_audio(path, target_sr=32000):
    # Load audio using librosa
    waveform, sr = librosa.load(path, sr=target_sr, mono=True)
    
    # Truncate or pad to target_length
    if len(waveform) > target_length:
        waveform = waveform[:target_length]
    elif len(waveform) < target_length:
        padding = torch.zeros(target_length - len(waveform))
        waveform = torch.cat([torch.tensor(waveform), padding])
    
    return waveform

# Load all audios and stack into batch tensor of shape [batch, samples]
audio_list = [load_audio(p, target_sr) for p in wav_paths]
audio_batch = torch.stack(audio_list)

print(f"Audio batch shape: {audio_batch.shape}")  # Should be [batch_size, 320000]

# Load PaSST model in 'embed_only' mode
model = get_basic_model(mode="embed_only")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Move batch to device
audio_batch = audio_batch.to(device)

# Forward pass to get embeddings
with torch.no_grad():
    embeddings = model(audio_batch)

print(f"Embeddings shape: {embeddings.shape}")


TypeError: expected Tensor as element 0 in argument 0, but got numpy.ndarray

In [13]:
import torch
import librosa
from hear21passt.base import get_basic_model

# Your .wav file paths
wav_paths = [
    '/home/teaching/DANN-pytorch/239946__rainbowdasheve__ghost.wav',
    '/home/teaching/DANN-pytorch/276470__unkleceeg__birds-chirping-03-short.wav',
    '/home/teaching/DANN-pytorch/182501__swiftoid__birds-chirping-03.wav'
]

# Parameters
target_sr = 32000
target_length = target_sr * 10  # 10 seconds * 32000 samples/sec

def load_audio(path, target_sr=32000):
    # Load audio with librosa (mono=True ensures single channel)
    waveform, sr = librosa.load(path, sr=target_sr, mono=True)

    # Convert to PyTorch tensor
    waveform = torch.tensor(waveform, dtype=torch.float32)

    # Pad or truncate to fixed length
    if waveform.shape[0] < target_length:
        padding = torch.zeros(target_length - waveform.shape[0], dtype=torch.float32)
        waveform = torch.cat([waveform, padding])
    else:
        waveform = waveform[:target_length]

    return waveform

# Load and process all audio files
audio_list = [load_audio(p, target_sr) for p in wav_paths]
audio_batch = torch.stack(audio_list)  # Shape: [batch_size, 320000]
print(f"Audio batch shape: {audio_batch.shape}")

# Load PaSST model in 'embed_only' mode
model = get_basic_model(mode="embed_only")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Move audio to device
audio_batch = audio_batch.to(device)

# Forward pass to extract embeddings
with torch.no_grad():
    embeddings = model(audio_batch)

print(f"Embeddings shape: {embeddings.shape}")  # Expected: [batch_size, embedding_dim]


Audio batch shape: torch.Size([3, 320000])


 Loading PASST TRAINED ON AUDISET 


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1):

In [12]:
print(f"Embeddings shape: {embeddings.shape}")  # Expected: [batch_size, embedding_dim]
print("Embeddings:\n", embeddings)


Embeddings shape: torch.Size([3, 768])
Embeddings:
 tensor([[ 0.4242,  0.8466, -0.5205,  ...,  0.2797,  0.8814, -1.3569],
        [-1.3094,  0.8903,  0.0708,  ...,  0.8115, -0.4800, -0.5528],
        [-1.3966,  0.6613, -0.3923,  ..., -0.5098,  0.1874, -0.1976]],
       device='cuda:0')


In [14]:
import torch
import librosa
from hear21passt.base import get_basic_model

# Path to a single audio file
wav_path = '/home/teaching/DANN-pytorch/239946__rainbowdasheve__ghost.wav'

# Parameters
target_sr = 32000
target_length = target_sr * 10  # 10 seconds

# Load and preprocess the audio
waveform, sr = librosa.load(wav_path, sr=target_sr, mono=True)
waveform = torch.tensor(waveform, dtype=torch.float32)

# Pad or truncate
if waveform.shape[0] < target_length:
    padding = torch.zeros(target_length - waveform.shape[0], dtype=torch.float32)
    waveform = torch.cat([waveform, padding])
else:
    waveform = waveform[:target_length]

# Add batch dimension: [1, samples]
audio_batch = waveform.unsqueeze(0)

# Load PaSST model in 'embed_only' mode
model = get_basic_model(mode="embed_only")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
audio_batch = audio_batch.to(device)

# Extract embedding
with torch.no_grad():
    embedding = model(audio_batch)  # shape: [1, 768]

print(f"Embedding shape: {embedding.shape}")
print(f"Embedding vector:\n{embedding.squeeze(0)}")




 Loading PASST TRAINED ON AUDISET 


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
    (1): Block(
      (norm1): LayerNorm((768,), ep

In [15]:
print(f"Embedding shape: {embedding.shape}")

Embedding shape: torch.Size([1, 768])
